In [ ]:
import os
import base64
from openai import OpenAI
from docai import utils

env = {
    "SCHEMA_TABLE_PARAMETER_NAME": "/dev/table/schema_table_name",
    "OPENAI_API_KEY_PARAMETER_NAME": "/dev/secret/openai/api_key"
}
os.environ.update(env)
resource = utils.Resources()
secrets = utils.Secrets()
schema_table = resource.get_table("SCHEMA_TABLE_PARAMETER_NAME")
client = OpenAI(api_key=secrets("OPENAI_API_KEY_PARAMETER_NAME"))

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
def get_tools():
    item = schema_table.scan()["Items"][5]
    schema = item["schema_definition"]
    schema.pop("description")
    # schema.pop("title")
    schema.pop("$schema")
    # schema.pop("$id")
    description = item["schema_description"]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_letter_of_employment_data",
                "description": description,
                "parameters": schema
            }
        },
    ]
    return tools

In [ ]:
data = open("../samples/media/loe.txt").read()
user_content = [
    {"type": "text", "text": f"I need help extracting data from an employment letter. \n\n Employment Letter: \n\n {data}"},
]
messages = [
    {"role": "system", "content": "Your are a data extracting assistant that can help humans extract data from a document."},
    {"role": "user", "content": user_content}
]
tools = get_tools()

In [ ]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    tools=tools,
)

In [ ]:
response.choices[0].message.json()

In [ ]:
tools